# 安装
```
pip install mako
```

# hello World

In [6]:
from mako.template import Template

templ = Template("hello world")
print  templ.render()


hello world


In [33]:
from mako.template import Template

print Template("hello ${data}").render(data="world")

hello world


# 基础用法

从文件

In [ ]:
from mako.template import Template

mytemplate = Template(filename='/docs/mytmpl.txt')
print(mytemplate.render())

In [ ]:
from mako.template import Template

mytemplate = Template(filename='/docs/mytmpl.txt', module_directory='/tmp/mako_modules')
print(mytemplate.render())

# 语法

## 表达式替换

最简单表达式就是变量替换．其语法就是``${}``.
```
this is ${x}
```
以上代码中, x的字符串表示形式将被应用到模板的输出流中．x的值通常来源于提供给模板rendering函数作为参数的Context变量. 如果x的值为给指定的模板，并且也未指定为局部变量,则会作为一改特殊的UNDEFINED.

``${}``中的内容会被python执行，所以各种表达式都是支持的.

```
${pow(x, 2) + pow(y, 2)}
```
在render到输出流之前，表达式的结果总是被估算为一个字符串.


In [38]:
from mako.template import Template
from mako.runtime import Context
from StringIO import StringIO

print Template("hello ${data}").render(data="world")

mytemplate = Template("hello, ${name}!")
buf = StringIO()
ctx = Context(buf, name="jack")
mytemplate.render_context(ctx)
print(buf.getvalue())


hello world
hello, jack!


## 表达式转义

mako包含了一系列的转义机制，包括HTML, URI和XML转义，以及``trim``函数.转义的动作可以通过``|``运算符附加到表达式后面:
```
${"this is some text"|u}
```
[!过滤器](http://docs.makotemplates.org/en/latest/filtering.html)



In [41]:
from mako.template import Template

mytemplate = Template('${"this is some text"|u}')
mytemplate.render()

mytemplate = Template('${" <tag>some value</tag> " | h,trim}')
mytemplate.render()

u'&lt;tag&gt;some value&lt;/tag&gt;'

## 控制结构
控制结构包括 ``if/eslse  while for try/except``等.
在mako中，控制语句用``%``后附加常规的``python``控制结构写法,并用``end<name>``结束语句.
其中``"<name>"``是该表达式的关键字.


``%``可以出现在一行的任何位置，只要其前面没有其他的文本．缩进是无所谓的.python的所有的"冒号"表达式在这里都是

In [57]:
from mako.template import Template

py = "% if 40>=5:\n"\
"    this is  a if \n "\
"% endif"
print py
mytemplate = Template(py)
mytemplate.render()


% if 40>=5:
    this is  a if 
 % endif


u'    this is  a if \n'

## 注释

注释有两种.
单行注释``##``
多行注释``<%doc> 文本 </%doc>``

In [76]:
from mako.template import Template

py1 = "## this a if \n" \
"% if 40>=5:\n"\
"    this is  signel \n "\
"% endif"

a = Template(py1)
a.render()

u'    this is  signel \n'

In [77]:
from mako.template import Template
py = "<%doc>\n" \
        "this a if \n" \
    "</%doc>\n" \
"% if 60>=5:\n"\
"    this is  a multi \n "\
"% endif"
m = Template(py)
m.render()

u'\n    this is  a multi \n'

## 模块级的语句块
``<% %>``的一个变体是<%! %>,代表模块级别的代码块．其中的代码会在模块级别执行，而不在模块的rendering函数中.所以，这段代码不能访问模板的context, 仅在模块加载到内存时被执行．
可以使用 ``<%! %>``来定义模板的导入语句.

在模板中，可以在任意位置,定义任意数量的<%! %>语句块．他们在编译产生的模块文件中出现的次序和定义时相同．

In [ ]:
<%!
    import mylib
    import re

    def filter(text):
        return re.sub(r'^@', '', text)
%>

## 标签

Mako 还提供了标签。所有标签的语法都一样，类似于 XML 标签，不同之处在于其标签名称必须以 ``% ``开头。标签的关闭可以用反斜杠的内联形式，或者独立的关闭标签：

```
<%include file="foo.txt"/>

<%def name="foo" buffered="True">
    this is a def
</%def>

```

每种标签都定义了一系列特定的属性。有些属性是必须的。并且，很多属性支持估算操作``(evaluation)``，也就是说你可以在属性文本中嵌入一个表达式（用 ``${} ``语法）。

```
<%include file="/foo/bar/${myfile}.txt"/>
```

属性是否支持运算时的估算，取决于标签的类型.


### ``<%page>``标签
该标签定义了模版的一些通用特性，包括缓存参数和模版调用的参数可选列表。
```
<%page args="x, y, z='default'"/>
```
也可以定义缓存特性
```
<%page cached="True" cache_type="memory"/>
```

目前每个模版只有一个``<%page>``标签生效。以后会改进。请确保每个模版中只用了一个。

 ```<%page> 的用途参考 The body() Method 和 Caching``` 

### ``<%include>``标签
和其他模版语言类似的标签，可以接受一个文件参数，调用那个文件的渲染效果：
```
<%include file="header.html"/>
    hello world
<%include file="footer.html"/>
```

还可以接受``<%page>``标签的参数，应用到导入的模版上：

```

<%includeinclude  file=file="toolbar.html""toolbar.html"  args=args="current_section='members', username='ed'""current_section= />
```


### ``<%def>``
定义了一个Python函数，函数可以在模版的其他位置调用。基本思想很简单。
``<%def>``标签比python的def强大的多，因为Mako提供了很多额外的功能，比如能将函数作为模版的"方法"，自动传递当前的Context,使用 缓冲/过滤/缓存标志，作为参数传递给其他的函数调用。 细节见 ``def and block``

In [ ]:
<%def name="myfunc(x)">
    this is myfunc, x is ${x}
</%def>

${myfunc(7)}

### ``<%block>``
和``<%def>``类似，但是会在根作用域立即执行，而且可以时匿名的：
```
<%block filter="h">
    some <html> stuff.
</%block>
```
借鉴了Jinjia2的块，有名字的块实现了一种很方便的继承方法：

```
<html>
    <body>
    <%block name="header">
        <h2><%block name="title"/></h2>
    </%block>
    ${self.body()}
    </body>
</html>
```

参考``Using Blocks``和``inheritance``


### ``<%namespace>``
Mako中的namespace和python中的import 语句等价，通过它可以访问其他模版的所有的渲染函数和metedata,python模块，还有本地定义的函数"包"。

```
<%namespace file="functions.html" import="*"/>

```

``%namespace`` 生成的底层对象是一个 ``mako.runtime.Namespace`` 实例。它是模板中的一个核心结构，用来引用模板的特定信息，比如当前的URI、继承结构和其他一些东西。名字空间见 Namespaces 。

### ``<%inherit>``
通过继承可以实现模版的继承链。它和其他模版语言的概念类似。

```
<%inherit file="base.html"/>
```

当使用``<%inherit>``标签时，控制首先被传递到继承模版的顶层模版，由他来决定如何处理调用部分。Mako在这部分实现的非常灵活，包括动态的继承，内容封装，多态方法调用。
参考 ``inheritance``


### <%nsname:defname>
可以通过 ``<%<namespacename>:<defname>>`` 来在一个名字空间中自定义一个标签。它的单标签和双标签形式分别对应行内表达式和 ``<%call>`` 标签.
```
<%mynamespace:somedef param="some value">
    this is the body
</%mynamespace:somedef>
```

如何自定义标签，参考``Calling a Def with Embedded Content and/or Other Defs ``

### <%call>

``%call`` 标签是用户定义标签的传统形式，大致和上面的 ``<%namespacename:defname> ``等价。该标签也参考 ``Calling a Def with Embedded Content and/or Other Defs ``。


### <%dco>
多行备注
```
<%doc>
    these are comments
    more comments
</%doc>

```

### <%text>
该标签使Mako词法分析器跳过该部分的处理，返回整个内容。主要用于写Mako的文档：
```
<%text filter="h">
    heres some fake mako ${syntax}
    <%def name="x()">${x}</%def>
</%text>
```


## 从模版提前返回
有时你想在模板或 ``<%def> ``方法的中间停止处理，只用当前得到的结果。可以在Python代码块中使用 return 语句来实现。
```
% if not len(records):
    No records found.
    <% return %>
% endif

```
或
```
<%
    if not len(records):
        return
%>
```